# IBM Ponder This - November 2024

## Problem Statement

A tetrahedron is a polyhedron composed of four triangular faces, six straight edges, and four vertices. We can write the lengths of the edges as a list, e.g., [2,2,2,2,2,2].

The volume $V$ of a tetrahedron with edge lengths [2,2,2,2,2,2] is an irrational number. However, setting the constant $c=144$, we find $cV^2=128$.

In the following, we keep $c=144$; do not pass another value of $c$ as part of your solution.

Your goal: Find a tetrahedron with integer edge lengths different than [2,2,2,2,2,2], which also has a volume $V$ satisfying $cV^2=128$. Provide your solution as the list of edge lengths.

A bonus "*" will be given for finding a tetrahedron with integer edge lengths satisfying $cV^2=655$.

## Solution

The volume of a tetrahedron can be computed from the edge lengths using the Cayley-Menger determinant. Assume a tetrahedron has vertices $A, B, C, D$ where $\triangle ABC$ is the base and $D$ is the apex. Let $a, b, c, d, e, f$ be the distances $AB$, $AC$, $BC$, $AD$, $BD$, $CD$, respectively. The volume of such a tetrahedron, $V$, satisfies

\begin{equation}
    288 V^2 = 
        \begin{vmatrix}
            0 & 1 & 1 & 1 & 1 \\
            1 & 0 & a^2 & b^2 & d^2 \\
            1 & a^2 & 0 & c^2 & e^2 \\
            1 & b^2 & c^2 & 0 & f^2 \\
            1 & d^2 & e^2 & f^2 & 0
        \end{vmatrix}.
\end{equation}

We can rewrite this as 

\begin{equation}
    V = \frac{1}{12} \sqrt{a^2 f^2 \alpha + b^2 e^2 \beta + c^2 d^2 \gamma - \rho}
\end{equation}

where

\begin{align}
    \alpha &= b^2 + e^2 + c^2 + d^2 - a^2 - f^2, \\
    \beta &= a^2 + f^2 + c^2 + d^2 - b^2 - e^2, \\
    \gamma &= a^2 + f^2 + b^2 + e^2 - c^2 - d^2, \\
    \rho &= a^2 b^2 c^2 + a^2 e^2 d^2 + f^2 b^2 d^2 + f^2 e^2 c^2.
\end{align}

Let $V^\prime = 144V^2$, we have

\begin{equation}
    V^\prime = a^2 f^2 \alpha + b^2 e^2 \beta + c^2 d^2 \gamma - \rho.
\end{equation}

One solution to solve the problem is to plug different edge lengths in the formula above until we find our solution. The issue is that the search space may be pretty large. We need to find ways to prune it.

The first observation is that the faces of the tetrahedrons are triangles and their side lengths must satisfy the triangle inequality which states that no side of the triangle should be longer than the sum of the other two. This allows to prune the search space a bit as we will exclude the invalid values for the third edge given the two first.

Another observation is that the target values for $V^\prime$ are very small, particularly when considering tetrahedrons with some larger edge lengths. Therefore, the tetrahedrons we are looking for are likely to be relatively flat. Assume we have two triangles sharing the same base which may not have the same orientation. If we connect the apices of those two triangles, we obtain a flat tetrahedron with volume zero. Therefore, given the 5 edge lengths of those triangles, we can compute the length of the remaining edge such that the tetrahedron have volume zero. Note that there are two such tetrahedrons (one when the apices of both triangles are above the base and another one when one of the apices is below the base).

Say the length of the shared base of the two triangles is $a$, the first triangle have other edge lengths $b$ and $c$ and the second triangle has other edge lengths $d$ and $e$. The coordinates of the apices of those triangles can be defined as 

\begin{align}
    (x_1, y_1) &= \left(\eta_1, \sqrt{b^2 - \eta_1^2} \right), \\
    (x_2, y_2) &= \left(\eta_2, \pm \sqrt{d^2 - \eta_2^2} \right),
\end{align}

where

\begin{align*}
\eta_1 &= \frac{a^2 + b^2 - c^2}{2a}, \\
\eta_2 &= \frac{a^2 + d^2 - e^2}{2a}.
\end{align*}

Let $f_1$ and $f_2$ be the last edge length in the two possible flat tetrahedrons. Those are the euclidean distances between the apices of the triangles which can be computed as

\begin{align}
    f_1 &= \sqrt{(\eta_2 - \eta_1)^2 + \left( \sqrt{d^2 - \eta_2^2} - \sqrt{b^2 - \eta_1^2} \right)^2}, \\
    f_2 &= \sqrt{(\eta_2 - \eta_1)^2 + \left( \sqrt{d^2 - \eta_2^2} + \sqrt{b^2 - \eta_1^2} \right)^2}.
\end{align}

This can be used to further reduce the search space. Given the 5 first edge lengths, we can compute $f_1$ and $f_2$. As we need integer lengths we round $f_1$ up and round $f_2$ down. Then we can compute the $V^\prime$ from those edge lengths. While the volume computed with $f_1$ ($f_2$) is lower than our target value, we increase $f_1$ (decrease $f_2$) by one. As soon as the volume becomes greater than our target value, we can stop the search using the 5 first edges. 

In order to speed up the search, we avoid some redundant computations. For example, we take $c$ to be always larger or equal to $b$ to avoid computing the same thing twice (when $b = x$, $c = y$ and $b = y$, $c = x$). Note that we could further optimise this by considering faces of the tetrahedrons. Here some interchangeable tetrahedrons are computed when the same triangles switch positions for the faces of the tetrahedron. 

Also, we set a maximum size on the edges of the tetrahedrons. It is set arbitrarily and increased if no solution is found.

Finally, we use numba's just-in-time compiler to speed up the whole process.

In [1]:
import numpy as np
from numba import njit


@njit
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)


@njit
def compute_distance_between_apex(a, b, c, d, e):
    eta_a = (a**2 + b**2 - c**2) / (2 * a)
    eta_b = (a**2 + d**2 - e**2) / (2 * a)

    coords1 = (eta_a, np.sqrt(b**2 - eta_a**2))
    coords2 = (eta_b, np.sqrt(d**2 - eta_b**2))

    return euclidean_distance(coords1[0], coords1[1], coords2[0], coords2[1]), euclidean_distance(coords1[0], coords1[1], coords2[0], -coords2[1])


@njit
def modified_cayley_menger_volume(a, b, c, d, e, f):
    alpha = b**2 + e**2 + c**2 + d**2 - a**2 - f**2
    beta = a**2 + f**2 + c**2 + d**2 - b**2 - e**2
    gamma = a**2 + f**2 + b**2 + e**2 - c**2 - d**2
    rho = a**2 * b**2 * c**2 + a**2 * e**2 * d**2 + f**2 * b**2 * d**2 + f**2 * e**2 * c**2
    return a**2 * f**2 * alpha + b**2 * e**2 * beta + c**2 * d**2 * gamma - rho


@njit
def f(limit):
    found128 = False
    found655 = False
    for a in range(1, limit + 1):
        for b in range(1, limit + 1):
            for c in range(max(abs(a - b) + 1, b), min(a + b, limit + 1)):
                for d in range(1, limit + 1):
                    for e in range(abs(a - d) + 1, min(a + d, limit + 1)):
                        f1, f2 = compute_distance_between_apex(a, b, c, d, e)
                        if not np.isnan(f1):
                            f1 = int(np.ceil(f1))
                            v1 = modified_cayley_menger_volume(a, b, c, d, e, f1)
                            while not found128 and v1 < 128:
                                f1 += 1
                                v1 = modified_cayley_menger_volume(a, b, c, d, e, f1)
                            if v1 == 128:
                                if a != 2:
                                    found128 = True
                                    print(f'cV^2 = 128   -->   [{a}, {b}, {c}, {d}, {e}, {f1}]')
                            while not found655 and v1 < 655:
                                f1 += 1
                                v1 = modified_cayley_menger_volume(a, b, c, d, e, f1)
                            if v1 == 655:
                                found655 = True
                                print(f'cV^2 = 655   -->   [{a}, {b}, {c}, {d}, {e}, {f1}]')
                        if not np.isnan(f2):
                            f2 = int(np.floor(f2))
                            v2 = modified_cayley_menger_volume(a, b, c, d, e, f2)
                            while not found128 and v2 < 128:
                                f2 -= 1
                                v2 = modified_cayley_menger_volume(a, b, c, d, e, f2)
                            if v2 == 128:
                                if a != 2:
                                    found128 = True
                                    print(f'cV^2 = 128   -->   [{a}, {b}, {c}, {d}, {e}, {f2}]')
                            while not found655 and v2 < 655:
                                f2 -= 1
                                v2 = modified_cayley_menger_volume(a, b, c, d, e, f2)
                            if v2 == 655:
                                found655 = True
                                print(f'cV^2 = 655   -->   [{a}, {b}, {c}, {d}, {e}, {f2}]')
                        if found655 and found128:
                            return None

In [2]:
f(300)

cV^2 = 655   -->   [9, 94, 102, 252, 245, 345]
cV^2 = 655   -->   [9, 245, 252, 102, 94, 345]
cV^2 = 128   -->   [69, 177, 245, 217, 280, 137]
